In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import plotly.express as px
from sklearn.decomposition import PCA
# from bioinfokit.visuz import cluster


In [2]:
!pip install bioinfokit

     |████████████████████████████████| 84 kB 2.0 MB/s 
  Created wheel for bioinfokit: filename=bioinfokit-2.0.8-py3-none-any.whl size=56750 sha256=c290dfeac4e0b1fa4d3684cae12b096bd444fe1beb75d6f099860b1387cd2647
  Stored in directory: /root/.cache/pip/wheels/0d/b0/a4/2be59655e33c889f2197d43e99ed077f05a6559e24ce28e21a
  Created wheel for adjustText: filename=adjustText-0.7.3-py3-none-any.whl size=7097 sha256=eebee2e39ab83ec9786d68fb45ce51012bf0d434ad3a448d396d4198bc13ead0
  Stored in directory: /root/.cache/pip/wheels/2f/98/32/afbf902d8f040fadfdf0a44357e4ab750afe165d873bf5893d
Successfully built bioinfokit adjustText


In [3]:
from bioinfokit.visuz import cluster

In [6]:
data = pd.read_csv('/content/drive/MyDrive/Academics/255/Project/Clean_Crime_Incidents.csv')
copy_of_data = data.copy(deep=True)
data.columns

Index(['Unnamed: 0', 'index', 'Case Number', 'Incident Datetime',
       'Incident Type Primary', 'Incident Description', 'Parent Incident Type',
       'Hour of Day', 'Day of Week', 'Address', 'Location', 'Latitude',
       'Longitude', 'Created At', '2010 Census Tract',
       '2010 Census Block Group', '2010 Census Block', 'Census Tract',
       'Census Block', 'Census Block Group', 'Neighborhood', 'Police District',
       'Council District', 'TRACTCE20', 'GEOID20_tract', 'GEOID20_blockgroup',
       'GEOID20_block'],
      dtype='object')

In [7]:
extracted_data = data[['Hour of Day', 'Census Tract', 'Neighborhood', 'Address', 'Latitude', 'Longitude', 'Parent Incident Type']]
extracted_data

,Hour of Day,Census Tract,Neighborhood,Address,Latitude,Longitude,Parent Incident Type
0,17,27.03,Broadway Fillmore,100 Block BECK ST,42.897,-78.838,Theft
1,11,71.04,Lower West Side,400 Block NIAGARA ST,42.896,-78.886,Assault
2,10,10.00,Seneca-Cazenovia,2300 Block SENECA ST,42.850,-78.803,Assault
3,23,37.00,Schiller Park,0 Block THEODORE ST,42.911,-78.808,Assault
4,2,164.00,Ellicott,FULTON ST & CHERRY ST,42.874,-78.871,Robbery
...,...,...,...,...,...,...,...
276375,0,28.01,Genesee-Moselle,100 Block SCHUTRUM ST,42.900,-78.818,Assault
276376,9,63.01,Elmwood Bidwell,1000 Block ELMWOOD AV,42.924,-78.877,Robbery
276377,16,23.00,Lovejoy,1000 Block E LOVEJOY ST,42.890,-78.809,Theft
276378,18,33.02,Masten Park,0 Block CELTIC PL,42.911,-78.847,Theft


In [8]:
data['Hour of Day'] =  data['Hour of Day'].astype(int)
data['Census Tract'] =  data['Census Tract'].astype(int)

In [9]:
# Label Encoding
labelEncoder = LabelEncoder()
extracted_data['Neighborhood'] = labelEncoder.fit_transform(extracted_data['Neighborhood'])
extracted_data['Address'] = labelEncoder.fit_transform(extracted_data['Address'])
extracted_data['Parent Incident Type'] = labelEncoder.fit_transform(extracted_data['Parent Incident Type'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

In [10]:
extracted_data

,Hour of Day,Census Tract,Neighborhood,Address,Latitude,Longitude,Parent Incident Type
0,17,27.03,2,5046,42.897,-78.838,7
1,11,71.04,20,9658,42.896,-78.886,0
2,10,10.00,29,8488,42.850,-78.803,0
3,23,37.00,27,1075,42.911,-78.808,0
4,2,164.00,6,14108,42.874,-78.871,4
...,...,...,...,...,...,...,...
276375,0,28.01,12,6296,42.900,-78.818,0
276376,9,63.01,7,6708,42.924,-78.877,4
276377,16,23.00,19,6699,42.890,-78.809,7
276378,18,33.02,22,189,42.911,-78.847,7


In [11]:
scaler = StandardScaler()
scaler.fit(extracted_data)
scaled_data = scaler.transform(extracted_data)
scaled_data

array([[ 7.01558766e-01, -1.40133596e-01, -1.44040038e+00, ...,
        -5.19302690e-01,  3.70844302e-01,  8.64825879e-01],
       [-1.11304777e-01, -1.01011332e-03,  2.75927874e-01, ...,
        -5.54641815e-01, -1.15832747e+00, -1.32543417e+00],
       [-2.46782035e-01, -1.93968473e-01,  1.13409200e+00, ...,
        -2.18024161e+00,  1.48586538e+00, -1.32543417e+00],
       ...,
       [ 5.66081509e-01, -1.52873147e-01,  1.80576304e-01, ...,
        -7.66676571e-01,  1.29471891e+00,  8.64825879e-01],
       [ 8.37036023e-01, -1.21198135e-01,  4.66631013e-01, ...,
        -2.45549259e-02,  8.41245960e-02,  8.64825879e-01],
       [-7.88691064e-01, -5.48766017e-02,  6.57334153e-01, ...,
         1.00027973e+00, -5.84888052e-01,  8.64825879e-01]])

In [ ]:

pca = PCA().fit(scaled_data)
# components = pca.fit_transform(scaled_data)
# print(pca)
pca_scores = PCA().fit_transform(scaled_data)
cluster.biplot(cscore=pca_scores, loadings=pca.components_, labels=extracted_data.columns.values, var1=round(pca.explained_variance_ratio_[0]*100, 2),var2=round(pca.explained_variance_ratio_[1]*100, 2))

# print(pca.explained_variance_ratio_[1]*100)
cluster.biplot(cscore=pca_scores, loadings=pca.components_, labels=extracted_data.columns.values, 
    var1=round(pca.explained_variance_ratio_[0]*100, 2), var2=round(pca.explained_variance_ratio_[1]*100, 2), 
    var3=round(pca.explained_variance_ratio_[2]*100, 2))



findfont: Font family ['Arial'] not found. Falling back to DejaVu Sans.
